In [1]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
from bs4 import BeautifulSoup
import re
from imblearn.over_sampling import SMOTE
import nlpaug.augmenter.word as naw
import random

2023-10-10 13:22:21.308068: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-10 13:22:22.015152: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roronoa/.conda/envs/tf/lib/python3.7/site-packages/nvidia/cudnn/lib:/home/roronoa/.conda/envs/tf/lib/:/home/roronoa/.conda/envs/tf/lib/python3.7/site-packages/nvidia/cudnn/lib:/home/roronoa/.conda/envs/tf/lib/:
2023-10-10 13:22:22.015255: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_pl

In [2]:
train_data_file = '../Data/Genre Classification Dataset/train_data.txt'
test_solution_file = '../Data/Genre Classification Dataset/test_data_solution.txt'

# Load the train data
train_df = pd.read_csv(train_data_file, delimiter=':::', names=['title', 'genre', 'desc'])

# Load the test data solution
test_solution_df = pd.read_csv(test_solution_file, delimiter=':::', names=['title', 'genre', 'desc'])

# Merge the train data and test data solution based on the "title" column
df = pd.concat([train_df, test_solution_df], axis=0, ignore_index=True)

/home/roronoa/.conda/envs/tf/lib/python3.7/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [3]:
del train_df
del test_solution_df


In [4]:
df.drop(columns='title', inplace=True)

In [5]:
df.isna().sum()

genre    0
desc     0
dtype: int64

In [6]:
df.duplicated().sum()

302

In [7]:
df = df.drop_duplicates(keep = "first")

In [8]:
df['desc'].iloc[23500]

" The story of Coral Fusion Latina; Canada's first and only Latin American choir. Its members are amateur singers and all are immigrants to Canada, representing countries such as Brazil, Columbia, Spain, Chile, Mexico, Costa Rica, Venezuela, and Argentina. The documentary follows the progress of the choir shortly after its inception, and culminates in their debut concert in front of a Vancouver audience one year later. Rehearsals, for the members of Fusion Latina, provide an opportunity to socialize with fellow Latinos and let loose in a way not afforded in their daily lives within Canada's relatively reserved Anglo-Saxon society. Their rambunctious behaviour, however, tend to turn rehearsals into free-for-all's, which frustrates their Argentinean born-Canadian raised conductor no end as she endeavors to prepare them for their debut."

In [9]:
df['genre'].value_counts()

 drama           27117
 documentary     26152
 comedy          14860
 short           10114
 horror           4406
 thriller         3179
 action           2626
 western          2061
 reality-tv       1763
 adventure        1548
 family           1546
 music            1437
 romance          1340
 sci-fi           1292
 adult            1179
 crime            1007
 animation         994
 sport             859
 talk-show         775
 fantasy           645
 mystery           636
 musical           548
 biography         529
 history           486
 game-show         387
 news              362
 war               264
Name: genre, dtype: int64

In [10]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['genre'] = encoder.fit_transform(df['genre'])

In [11]:
#Upon Checking the dataset, There seems to be no sign of anomalies like html tags, urls, etc
#For reassurance, we can run the code 1 time

In [12]:
def remove_urls(text):
    # Define a regular expression pattern to match URLs
    url_pattern = r'https?://\S+|www\.\S+'
    return re.sub(url_pattern, '', text)

df['desc']= df['desc'].apply(remove_urls)

In [13]:
#Removing HTML tags from the whole dataset
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

df['desc'] = df['desc'].apply(remove_html_tags)

/home/roronoa/.conda/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
# Since there was no space after each full stop, NLTK sent_tokenize considered 2 sentences as 1
def insert_space_after_full_stop(text):
    return text.replace('.', '. ')
df['desc'] = df['desc'].apply(insert_space_after_full_stop)

In [15]:
def lemmatization(text):
    lm= WordNetLemmatizer()
    text = ' '.join([lm.lemmatize(word, pos='v') for word in text.split()])
    return text
df['desc']= df['desc'].apply(lemmatization)
df.head()

,genre,desc
0,8,Listening in to a conversation between his doc...
1,24,A brother and sister with a past incestuous re...
2,1,As the bus empty the students for their field ...
3,8,"To help their unemployed father make end meet,..."
4,8,The film's title refer not only to the un-reco...


In [17]:
df.iloc[24]['desc']

"Dong-woo be release from prison after a short time follow his fail attempt to rob a watch shop. He decide to follow in his brother's footsteps and become a singer. His popularity soon eclipse his brother's, however, lead to unexpected jealousy and family problems"

In [19]:
#Applying major transformation to text such as lowering text, removing stopwords, stemming, removing
#punctuations

ps = PorterStemmer();
def text_transform(text):
    text= text.lower()
    text = nltk.word_tokenize(text)
    y=[]
    for i in text:
        if i.isalnum():
            y.append(i)
    
    text = y[:]
    y.clear()
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
    text = y[:]
    y.clear()
    for i in text:
        y.append(ps.stem(i))
        
    
    return " ".join(y)

In [20]:
df['desc']=df['desc'].apply(text_transform)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108112 entries, 0 to 108413
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   genre   108112 non-null  int64 
 1   desc    108112 non-null  object
dtypes: int64(1), object(1)
memory usage: 2.5+ MB


In [25]:
df.iloc[24]['desc']

'releas prison short time follow fail attempt rob watch shop decid follow brother footstep becom singer popular soon eclips brother howev lead unexpect jealousi famili problem'

In [28]:
df.to_csv('../Data/Movie_Genre(Dataset1).csv', index = False)